In [1]:
import time
from datasets import *
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from SOCT.LinearClassifierHeuristic import LinearClassifierHeuristic
from SOCT.SOCTStumpHeuristic import SOCTStumpHeuristic
from SOCT.SOCTFull import SOCTFull
from SOCT.SOCTBenders import SOCTBenders
from SOCT.utils import *

In [2]:
X, y = load_ionosphere()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train, X_test = preprocess_dataset(X_train, X_test, numerical_features=X_train.columns)
print("X_train dimensions:", X_train.shape)

max_depth = 4
heuristic_time_limit = 60
time_limit = 600

X_train dimensions: (263, 34)


In [2]:
X, y = load_tictactoe_endgame()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train, X_test = preprocess_dataset(X_train, X_test, categorical_features=X_train.columns)
print("X_train dimensions:", X_train.shape)

max_depth = 4
heuristic_time_limit = 60
time_limit = 600

X_train dimensions: (718, 27)


In [3]:
# S-OCT Benders without warm start
start_time = time.time()
clf = SOCTBenders(max_depth=max_depth, ccp_alpha=0.00001, time_limit=time_limit)
clf.fit(X_train, y_train)
end_time = time.time()
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
running_time = end_time - start_time
callback_time = clf.master_._callback_time
callback_frac = callback_time / running_time
print("Train/test accuracy, running time, callback calls/time/frac:", train_acc, test_acc, running_time, clf.master_._callback_calls, callback_time, callback_frac)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-10
Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 600
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 45250 rows, 33793 columns and 100552 nonzeros
Model fingerprint: 0x70e36574
Variable types: 22305 continuous, 11488 integer (11488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 11504 rows and 11504 columns
Presolve time: 0.11s
Presolved: 33746 rows, 22289 columns, 78980 nonzeros
Variable types: 11519 continuous, 10770 integer (10770 binary)
Found heuristic solution: objective -0.6573816

Root relaxation: objective -9.999950e-01, 15428 iterations, 0.37 seconds (0.79 work units)

    Nodes    |    Current Node    |     Objective Boun

In [4]:
# S-OCT stump heuristic
start_time = time.time()
clf = SOCTStumpHeuristic(max_depth=max_depth, time_limit=heuristic_time_limit)
clf.fit(X_train, y_train)
end_time = time.time()
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
running_time = end_time - start_time
print("Train/test accuracy, running time:", train_acc, test_acc, running_time)
warm_start = clf.branch_rules_, clf.classification_rules_

Train/test accuracy, running time: 1.0 0.9583333333333334 3.5306081771850586


In [5]:
# S-OCT Benders with S-OCT stump warm start
start_time = time.time()
clf = SOCTBenders(max_depth=max_depth, ccp_alpha=0.00001, warm_start_tree=warm_start, time_limit=time_limit-running_time)
clf.fit(X_train, y_train)
end_time = time.time()
train_acc = clf.score(X_train, y_train)
test_acc = clf.score(X_test, y_test)
running_time = end_time - start_time
callback_time = clf.master_._callback_time
callback_frac = callback_time / running_time
print("Train/test accuracy, running time, callback calls/time/frac:", train_acc, test_acc, running_time, clf.master_._callback_calls, callback_time, callback_frac)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 5.9646939182281494e+02
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 45250 rows, 33793 columns and 100552 nonzeros
Model fingerprint: 0xf25a6eb7
Variable types: 22305 continuous, 11488 integer (11488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Loaded user MIP start with objective -0.99998

Presolve removed 11504 rows and 11504 columns
Presolve time: 0.10s
Presolved: 33746 rows, 22289 columns, 78980 nonzeros
Variable types: 11519 continuous, 10770 integer (10770 binary)

Root relaxation: objective -9.999950e-01, 15428 iterations, 0.37 seconds (0.79 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Bes